In [1]:
import sys
sys.path.insert(0, '/home/khalil.nor/cantera/build/python')

In [2]:
import cantera as ct
import os

In [3]:
print("Running Cantera Version: " + str(ct.__version__))

Running Cantera Version: 2.6.0b1


In [ ]:
###need to specify python path in sbatch script when running over all flame speeds 

In [ ]:

#species and the stoic where equivalence ratio = 1
#species = {'C2H6': [0.025,0.1525,0.0566],'CH4': [0.05, 0.1525,0.095] ,'CH2F2': [0.11,0.225,0.174] ,'CH3F': [0.08,.225,0.123],'C2H5F': [0.025,0.150,0.0654],'CH2FCH2F' :[0.0375,0.150,0.0775],'CH2FCHF2' :[0.075,0.150,0.095],'CH3CF3' :[0.07,0.150,0.095],'CH3CHF2':[0.0375, 0.150,0.0775]}



species = {'C2H6': [0.025,0.1525,0.0566]}

#Constants
To = 298
Po = ct.one_atm

for label,value in species.items():
    #find all chem.cti files for each halocarbon/hydrocarbon. Evaluate at their specified equivalence ratios.
    directory = f'{label}/cantera/chem.cti'
    i = value[0]
    mole_frac_list = []
    
    while i < value[1]: 
      mole_frac_list.append(i)
      i += 0.0025 
        
    #can use below to only test one vol_frac
    mole_frac_list = [0.0566]
    
    gas = ct.Solution(directory)
    
    for mole in mole_frac_list: 
        print(mole)
        X_stoic = value[2]
        norm_ox = (1-X_stoic)*.21
        mole_frac_dict = {'C2H6(1)': (X_stoic/norm_ox), 'O2(2)':((1-X_stoic)*.21)/norm_ox, 'N2':((1-X_stoic)*0.79)/norm_ox }
        gas.TPX = To, Po, mole_frac_dict
        width = 0.08
        flame = ct.FreeFlame(gas, width=width)
        flame.set_refine_criteria(ratio=3, slope=0.1, curve=0.1)
        flame.max_time_step_count = 900
        loglevel = 1
    
   #Calculate Flame Speeds using 'Mixture-averaged' transport model 
        flame.transport_model='Mix'
        flame.solve(loglevel=loglevel, auto=True)
    
        print('mixture-averaged flamespeed at volume fraction {0:7f}= {0:7f} m/s'.format(mole,flame.velocity[0]))
    '''
    Su = flame.u[0]
    results[x] = Su
    sltn = flame.to_solution_array()
    pd = sltn.to_pandas()
    pd.to_csv(f'data/test_{x}.csv')

vol_fracs = list(results.keys())
flame_speeds = list(results.values())
    '''




    
'''


i = 0.08
mole_frac_list = []

while i < 0.225:
   mole_frac_list.append(i)
   i += 0.0025

#can use below to only test one vol_frac
#mole_frac_list = [0.123]


results = {}

for x in mole_frac_list:
    norm_ox = (1-x)*.21
    mole_frac_dict = {'CH3F(1)': (x/norm_ox), 'O2(2)':((1-x)*.21)/norm_ox, 'N2':((1-x)*0.79)/norm_ox }
    gas.TPX = To, Po, mole_frac_dict
    width = 0.08
    flame = ct.FreeFlame(gas, width=width)
    flame.set_refine_criteria(ratio=3, slope=0.1, curve=0.1)
    flame.max_time_step_count = 900
    loglevel = 1
    flame.solve(loglevel=loglevel, auto=True)
    Su = flame.u[0]
    results[x] = Su
    sltn = flame.to_solution_array()
    pd = sltn.to_pandas()
    pd.to_csv(f'data/test_{x}.csv')

vol_fracs = list(results.keys())
flame_speeds = list(results.values())


print("volume fractions are:")
print(vol_fracs)

print("flame speeds are:")
print(flame_speeds)

#print(xresults)
print("volume fractions list is " + str(mole_frac_list))
'''

/tmp/ipykernel_3833/1811946274.py:25: DeprecationWarning: XML_Node::build: 
The CTI and XML input file formats are deprecated and will be removed in
Cantera 3.0. Use 'cti2yaml.py' or 'ctml2yaml.py' to convert CTI or XML input
files to the YAML format. See https://cantera.org/tutorials/legacy2yaml.html
for more information.
  gas = ct.Solution(directory)


0.0566

************ Solving on 8 point grid with energy equation enabled ************

..............................................................................
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     1.068e-05      4.058
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     3.421e-05       6.31
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     2.435e-05      6.216
Attempt Newton solution of steady-state problem...    success.

Problem solved on [9] point grid(s).

..............................................................................
grid refinement disabled.

******************** Solving with grid refinement enabled ********************

..............................................................................
Attempt Newton solution of steady-state problem...    success.

Problem solved on [9] point grid(s).

...........................................

In [ ]:
gas = ct.Solution('C2H6/cantera/chem.cti')

In [6]:
print('mixture-averaged flamespeed = {0:7f} m/s'.format(flame.velocity[0]))

mixture-averaged flamespeed = 0.423174 m/s


In [ ]:
gas.binary_diff_coeffs
